In [2]:
import numpy as np

def detect_outliers_zscore(data, threshold=3):
    mean = np.mean(data)
    std = np.std(data)
    z_scores = [(y - mean) / std for y in data]
    print('ran')
    return np.where(np.abs(z_scores) > threshold)


In [3]:
def detect_outliers_iqr(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    outlier_indices = np.where((data < Q1 - 1.5 * IQR) | (data > Q3 + 1.5 * IQR))
    print('ran')
    return outlier_indices


In [4]:
from sklearn.ensemble import IsolationForest
import numpy as np

def detect_outliers_isolation_forest(data):
    # Reshape data if it's a 1D array
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    
    clf = IsolationForest(random_state=42, contamination='auto')
    preds = clf.fit_predict(data)
    
    # Find indices of outliers
    outlier_indices = np.where(preds == -1)
    print('ran')
    return outlier_indices


In [5]:
from sklearn.neighbors import LocalOutlierFactor

def detect_outliers_lof(data):
    # Reshape data if it's a 1D array
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    lof = LocalOutlierFactor(n_neighbors=20, contamination='auto')
    preds = lof.fit_predict(data)
    outlier_indices = np.where(preds == -1)
    print('ran')
    return outlier_indices


In [6]:
from sklearn.cluster import DBSCAN

def detect_outliers_dbscan(data, eps=0.5, min_samples=5):
    # Reshape data if it's a 1D array
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    preds = dbscan.fit_predict(data)
    outlier_indices = np.where(preds == -1)
    print('ran')
    return outlier_indices


In [7]:
from sklearn.svm import OneClassSVM

def detect_outliers_one_class_svm(data, nu=0.05, gamma='scale'):
    # Reshape data if it's a 1D array
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    oc_svm = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
    preds = oc_svm.fit_predict(data)
    outlier_indices = np.where(preds == -1)
    print('ran')
    return outlier_indices


In [10]:
import pandas as pd
import os
from pync import Notifier

df = pd.read_csv('../csv/merged_data.csv')

# Detect outliers using Z-Score
# target_feature = df['aqi'].values
# outliers_zscore = detect_outliers_zscore(target_feature)
# outliers_iqr = detect_outliers_iqr(target_feature)
# outliers_isolation_forest = detect_outliers_isolation_forest(target_feature)
# outliers_lof = detect_outliers_lof(target_feature)

# print("Outliers detected using Z-Score:", outliers_zscore)
# print("Outliers detected using IQR:", outliers_iqr)
# print("Outliers detected using Isolation Forest:", outliers_isolation_forest)
# print("Outliers detected using Local Outlier Factor:", outliers_lof)


df['date'] = pd.to_datetime(df[['year', 'month', 'day']])  # This combines year, month, day

# Placeholder for detected outlier dates
outlier_dates_set = set()
outlier_info = []

# Example usage on the 'value' column
methods = [detect_outliers_zscore, detect_outliers_iqr, detect_outliers_isolation_forest, detect_outliers_lof]

for method in methods:
    outlier_indices = method(df['aqi'].values)
    # Extracting the dates and AQI values of detected outliers
    outlier_dates_aqi = df.iloc[outlier_indices[0]][['date', 'aqi']]
    for _, row in outlier_dates_aqi.iterrows():
        outlier_info.append((row['date'], row['aqi']))

# Sort the outliers by date
outlier_info = sorted(outlier_info, key=lambda x: x[0])
#then by aqi
outlier_info_sorted_by_aqi = sorted(outlier_info, key=lambda x: x[1], reverse=True)[:10]

# Now, write the outlier dates and AQI values to a file
with open('outlier_dates_aqi.txt', 'w') as file:
    for date, aqi in outlier_info:
        file.write(f"Date: {date}, AQI: {aqi}\n")
    file.write("Top Ten AQI Dates:")
    for date, aqi in outlier_info_sorted_by_aqi:
        file.write(f"Date: {date}, AQI: {aqi}\n")

print("Outlier dates and AQI values saved to outlier_dates_aqi.txt")

/var/folders/6m/1wyk0cv146d91z1nsb6cxh6w0000gn/T/ipykernel_36191/1691395734.py:5: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../csv/merged_data.csv')


ran
ran


KeyboardInterrupt: 